In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [99]:
# В любой непонятной ситуации не забудь удалить все окна.
cap.release()
cv2.destroyAllWindows()

In [3]:
# Открываем видеопоток со стандартной камеры.
cap = cv2.VideoCapture(0)

# считаем 2 первых кадра
ret, frame1 = cap.read()
ret, frame2 = cap.read()

# параметры ShiTomasi для поиска углов
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# параметры lucas-kanade для отслеживания оптического потока
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Найдем начальные точки, которые будут отслеживаться
p0 = cv2.goodFeaturesToTrack(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), mask = None, **feature_params)
#p0 = np.float32(np.array([np.array([np.array([0, frame1.shape[1]//2])]), 
#               np.array([np.array([frame1.shape[0]//2, 0])]),
#               np.array([np.array([frame1.shape[0], frame1.shape[1]//2])]),
#               np.array([np.array([frame1.shape[0]//2, frame1.shape[1]])])]))
# Значения ширины и высоты смещения кадра
fr_w = 0
fr_h = 0
while cap.isOpened():
    # Поиск движущихся объектов
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    dilated = cv2.dilate(blur, None, iterations=3)
    _ , thresh = cv2.threshold(dilated, 20, 255, cv2.THRESH_BINARY)
    
    cntrs, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cntr in cntrs:
        if cv2.contourArea(cntr) < 500:
            continue
        (x, y, w, h) = cv2.boundingRect(cntr)
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Поиск движения камеры
    # Рассчитаем следующую точку для получения направления движения
    p1, st, err = cv2.calcOpticalFlowPyrLK(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY), p0, None, **lk_params)

    # Если отслеживаемые точки остались за кадром то найдем их снова
    if p1.shape[0] < 2:
        p0 = cv2.goodFeaturesToTrack(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), mask = None, **feature_params)
        fr_w = 0
        fr_h = 0
        p1, st, err = cv2.calcOpticalFlowPyrLK(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY), p0, None, **lk_params)
    # Если st==1, то было движение
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    # Рассчет новой ширины и высоты смещения кадра по смещению 1 точки из отслеживаемых
    fr_w += good_new[0][0] - good_old[0][0]
    fr_h += good_new[0][1] - good_old[0][1]
    #fr_w += (good_new[:, 0] - good_old[:, 0]).mean()
    #fr_h += (good_new[:, 1] - good_old[:, 1]).mean()
    # Нарисуем черные прямоугольники для компенсации кадра
    if fr_w >= 0:
        cv2.rectangle(frame1, (0, 0), (int(fr_w), frame1.shape[0]), (0, 0, 0), -1)
    else:
        cv2.rectangle(frame1, (frame1.shape[1] - int(-fr_w), 0), (frame1.shape[1], frame1.shape[0]), (0, 0, 0), -1)
    
    if fr_h >= 0:
        cv2.rectangle(frame1, (0, 0), (frame1.shape[1], int(fr_h)), (0, 0, 0), -1)
    else:
        cv2.rectangle(frame1, (0, frame1.shape[0] - int(-fr_h)), (frame1.shape[1], frame1.shape[0]), (0, 0, 0), -1)
    
    
    # Выведем видео
    cv2.imshow('Video', frame1)                
    
    frame1 = frame2
    ret, frame2 = cap.read()
    p0 = good_new.reshape(-1,1,2)
    
    # Крутимся в цикле, пока пользователь не нажмет Esc.
    # По нажатию Space фиксируем кадр для отслеживания движения камеры
    key = cv2.waitKey(1)
    if key == 27:  # Esc
        break
    elif key == 32: # Space
        p0 = cv2.goodFeaturesToTrack(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), mask = None, **feature_params)
        #p0 = np.float32(np.array([np.array([np.array([0, frame1.shape[1]//2])]), 
        #       np.array([np.array([frame1.shape[0]//2, 0])]),
        #       np.array([np.array([frame1.shape[0], frame1.shape[1]//2])]),
        #       np.array([np.array([frame1.shape[0]//2, frame1.shape[1]])])]))
        fr_w = 0
        fr_h = 0

# Отпускаем видеопоток, закрываем все окна.
# Если из-за ошибки мы сюда не доберемся, это надо сделать вручную, а то придется перезапускать ядро.
cap.release()
cv2.destroyAllWindows()

In [78]:
a = np.float32(np.array([np.array([np.array([0, frame1.shape[1]//2])]), 
                np.array([np.array([frame1.shape[0]//2, 0])]),
                np.array([np.array([frame1.shape[0], frame1.shape[1]//2])]),
                np.array([np.array([frame1.shape[0]//2, frame1.shape[1]])])]))
print(np.float32(a))

[[[  0. 320.]]

 [[240.   0.]]

 [[480. 320.]]

 [[240. 640.]]]


In [72]:
frame1.shape

(480, 640, 3)